# Loading in a user

In [1]:
import bandicoot as bc
user = bc.read_csv('ego', 'data/', 'data/antennas.csv')

[x] 314 records from 2014-03-02 07:13:30 to 2014-04-14 12:04:37
[x] 6 contacts
[ ] No attributes stored
[x] 27 antennas
[x] Has home
[x] Has texts
[x] Has calls


In [2]:
import pandas
from collections import Counter

In [3]:
df_antennas = pandas.DataFrame(map(lambda (a, b): (a, ) + b, user.antennas.items()))
df_antennas.columns = ['id', 'latitude', 'longitude']
df_antennas['interactions'] = map(lambda i: sum([1 for r in user.records if r.position.antenna == i]), df_antennas.id)
df_antennas.to_csv('viz/mobility_view/antenna.csv', index=False)

In [4]:
antennas_id = [r.position.antenna for r in user.records]
transitions = Counter([(min(i, j), max(i, j)) for (i, j) in zip(antennas_id, antennas_id[1:])])
df_transitions = pandas.DataFrame([a + (count, ) for a, count in transitions.items()])
df_transitions.columns = ['source', 'target', 'amount']
df_transitions = df_transitions[df_transitions.source != df_transitions.target]
df_transitions.to_csv('viz/mobility_view/transitions.csv', index=False)

In [5]:
%%html
<iframe src="viz/mobility_view/index.html" width="100%" height="400px">

## Event Timeline

In [6]:
timeline = pandas.DataFrame([(r.datetime, ('inc_' if r.direction == 'in' else 'out_') + r.interaction, r.call_duration) for r in user.records])
timeline.columns = ['time', 'type', 'call_duration']
timeline['time'] = timeline['time'].map(lambda x: x.strftime('%d-%m-%y %H:%M'))
timeline.to_csv('viz/event_timeline/timeseries.csv', index=False)

In [7]:
%%html
<iframe src="viz/event_timeline/index.html" width="100%" height="400px">

## Network view

In [8]:
m_texts = bc.network._interaction_matrix_text(user)
m_calls = bc.network._interaction_matrix_call(user)
nb_users = len(m_texts)

df_links = pandas.DataFrame([(i, j, m_calls[j][i], m_calls[i][j], m_texts[j][i], m_texts[i][j]) for i in range(nb_users) for j in range(nb_users)]).fillna(0)
df_links.columns = ['source', 'target', 'inc_calls', 'out_calls', 'inc_texts', 'out_texts']
df_links = df_links[df_links[['inc_calls', 'out_calls', 'inc_texts', 'out_texts']].sum(axis=1) > 0]
df_links = df_links[df_links.source < df_links.target]
df_links.to_csv('viz/network_view/links.csv', index=False)

In [9]:
nodes = set(df_links.source) | set(df_links.target)
pandas.DataFrame([int(None in m_texts[n] or None in m_calls[n]) for n in nodes], columns=['no_network_info']).to_csv('viz/network_view/nodes.csv', index=False)

In [10]:
%%html
<iframe src="viz/network_view/index.html" width="100%" height="400px">